In [4]:
import pandas as pd
import os

from sapsam import parser, constants  #, detector

import filter_JSON_for_event_labels

csv_paths = parser.get_csv_paths()

# df_raw=pd.read_csv(os.getcwd()+'/models/0.csv', sep=",")
csv_path = csv_paths[0]
print(csv_path.name)
df_raw = pd.read_csv(csv_path, nrows=200)

SyntaxError: invalid syntax (<ipython-input-4-8f076917e1ca>, line 4)

In [5]:
df = filter_JSON_for_event_labels.add_event_labels_to_DF(df_raw)
print(df.shape)
df.head(3)

NameError: name 'filter_JSON_for_event_labels' is not defined

In [ ]:
df.drop(columns=['Revision ID', 'Organization ID', 'Datetime', 'Model JSON', 'Description', 'Type', 'Namespace'], inplace=True)
df.dropna(subset=['event_labels'], inplace=True)

## Language Filter

In [ ]:
## !python -m spacy download en_core_web_sm

In [ ]:
import spacy
from spacy.language import Language
from spacy_langdetect import LanguageDetector
from tqdm import tqdm


def get_df_models_and_labels(df, sep_str=" "):
    df_labels = df.drop(columns="category", axis=1)
    df_labels.reset_index(inplace=True)
    df_labels.drop(columns="element_id", axis=1, inplace=True)
    df_labels.label = df_labels.label.apply(lambda x: str(x or ''))
    df_labels.drop_duplicates(ignore_index=True, inplace=True)
    df_labels['label'] = df_labels.groupby(['model_id'])['label'].transform(lambda x: sep_str.join(x))
    df_labels.drop_duplicates(ignore_index=True, inplace=True)
    return df_labels.set_index("model_id")


def clean(label):
    # handle some special cases
    label = label.replace("\n", " ").replace("\r", "")
    # delete unnecessary whitespaces
    label = label.strip()
    return label


def get_lang_detector(nlp, name):
    return LanguageDetector()


class ModelLanguageDetector:
    def __init__(self, threshold):
        self.threshold = threshold
        self.nlp = spacy.load("en_core_web_sm")
        Language.factory("language_detector", func=get_lang_detector)
        self.nlp.add_pipe('language_detector', last=True)

    def _get_text_language(self, text):
        doc = self.nlp(str(clean(text)))
        detection = doc._.language
        return detection['language']

    def add_detected_natural_language_from_meta(self, df_meta):
        df_meta['detected_natural_language'] = [self._get_text_language(name) for name in tqdm(df_meta.name)]

    def get_detected_natural_language_from_bpmn_model(self, series_txt):
 #       df_labels = get_df_models_and_labels(df, " ")
        # df['detected_natural_language'] = [self._get_text_language(label) for label in tqdm(df['event_labels'])]
        return series_txt.apply(self._get_text_language)


In [ ]:
ld = ModelLanguageDetector(0.8) # comment this line if you have already created the pkl file
df['detected_natural_language'] = ld.get_detected_natural_language_from_bpmn_model(df['event_labels']) # comment this line if you have already created the pkl file
# df.to_pickle(constants.DATA_INTERIM / "all_languages.pkl") # comment this line if you have already created the pkl file
# df = pd.read_pickle(constants.DATA_INTERIM / "bpmn_languages.pkl") # uncomment this line if you have already created the pkl file
df.head()

In [ ]:
df['detected_natural_language'].value_counts()


In [ ]:
df = df[df['detected_natural_language'].isin(['en','ca'])]
## todo: check all english language codes

In [ ]:
df.head()